# MANU 465 EEG Project
## Exploring Brainwaves vs Creative and Analytical Tasks
### By: Faith Tan, Emilie Ho, and Pan Tisapramotkul 

## Project Overview and Summary

### Import Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

### Constants used

In [104]:
### CONSTANTS ###

SAMPLING_RATE = 256
CREATIVE_DIR = 'dataset/Drawing/'
MATH_DIR = 'dataset/Mathematical/'

RAW_CHANNEL = ['RAW_AF7', 'RAW_AF8', 'RAW_TP9', 'RAW_TP10']

## Importing Raw Data Collected

In [105]:
import os

# Get all the files path in the mathematical and creative directories appened in a list
math_files = [os.path.join(MATH_DIR, file) for file in os.listdir(MATH_DIR) if os.path.isfile(os.path.join(MATH_DIR, file))]
creative_files = [os.path.join(CREATIVE_DIR, file) for file in os.listdir(CREATIVE_DIR) if os.path.isfile(os.path.join(CREATIVE_DIR, file))]

## Data Preprocessing

This is a SAMPLE way on how to perform FFT transform from the **raw data in microvolts** collected by MUSE Monitor

MUSE Monitor has 4 electrodes (TF9,AF7,AF8,TP10) that collects data as explained here: https://mind-monitor.com/Technical_Manual.php#help_graph_raw

The example below perform FFT on **AF7** data collected. 

After performing FFT, the peaks, amplitude, and area under the curve maybe be considered to use as features for ML models

In [ ]:
## PLOTTING THE FFT OF THE RAW_AF7 DATA ##
from scipy.signal import hilbert
from math import floor


def divide_data(data, size):
    #TODO: Maybe remove a few seconds at the begining and the end ? 

    # Split the DataFrame into chunks
    num_chunks = floor(len(data) / size)
    data_list = []  # List to store chunks

    for i in range(num_chunks):
        start_idx = i * size
        end_idx = start_idx + size
        data_list.append(data.iloc[start_idx:end_idx])

    return data_list

def cleaned_up_data(file : str):
    # get a dataframe
    dataset = pd.read_csv(file)
    
    # drop columns and NaN values
    dataset.drop(columns=['Battery','Elements'], inplace=True)
    dataset = dataset.dropna()

    dataset['Delta_Average'] = ((dataset['Delta_TP9'] + dataset['Delta_TP10'] + dataset['Delta_AF7'] + dataset['Delta_AF8']) / 4) * 100
    dataset['Gamma_Average'] = ((dataset['Gamma_TP9'] + dataset['Gamma_TP10'] + dataset['Gamma_AF7'] + dataset['Gamma_AF8']) / 4) * 100
    dataset['Theta_Average'] = ((dataset['Theta_TP9'] + dataset['Theta_TP10'] + dataset['Theta_AF7'] + dataset['Theta_AF8']) / 4) * 100
    dataset['Alpha_Average'] = ((dataset['Alpha_TP9'] + dataset['Alpha_TP10'] + dataset['Alpha_AF7'] + dataset['Alpha_AF8']) /4) * 100
    dataset['Beta_Average'] = ((dataset['Beta_TP9'] + dataset['Beta_TP10'] + dataset['Beta_AF7'] + dataset['Beta_AF8']) / 4 ) * 100
    
    dataset['delta_smooth_data'] = moving_average(dataset['Delta_Average'].values, 1000)
    dataset['delta_envelope'] = np.abs(hilbert(dataset['delta_smooth_data'].values))
    return dataset

def moving_average(data, window_size):
    return np.convolve(data, np.ones(window_size)/window_size, mode='same')

def get_all_data(file_list: list):
    """get data from all the files in the directory, 
    divide it into 20 seconds intervals and return the data"""

    all_data = []

    size = SAMPLING_RATE * 20

    for file in file_list:
        dataset = cleaned_up_data(file)
        data_list = divide_data(dataset, size=size)
        all_data.extend(data_list)
    
    return all_data

all_math_data = get_all_data(math_files)
all_drawing_data = get_all_data(creative_files)


(5120, 44)


## Feature Extraction

Get the peaks' amplitude, frequency, and area under the curve from the raw data collected by the four electrodes

In [ ]:
from scipy.signal import find_peaks
from numpy import trapezoid #type: ignore

def get_fft(dataset, channel : str):
    """Perform FFT on the data of the channel specfied from the range of 0.5 to 50 Hz

    Args:
        dataset (pd.DataFrame): _description_
        channel (str): _description_

    Returns:
        _type_: _description_
    """
    dataset = dataset[channel]

    # Perform FFT 
    n = len(dataset)                 # length of the signal
    k = np.arange(n)
    T = n/SAMPLING_RATE
    frq = k/T                 # two sides frequency range
    zz=int(n/2)

    freq = frq[range(zz)]           # one side frequency range
    Y = np.fft.fft(dataset)/n              # fft computing and normalization
    Y = abs(Y[range(zz)])

    # get only frequency from 0.5 to 50 Hz
    freq_mask = (freq>0.5) & (freq<50)
    filtered_freq = freq[freq_mask]
    filtered_Y = Y[freq_mask]
    
    return filtered_freq, filtered_Y
    
def get_peaks(filtered_freq, filtered_Y):
    if max(filtered_freq) > 60:
        print("Warning: Detected frequency Higher than 60 Hz. Please remove it before continuing")
        return 
    
    # Get the peaks
    peaks, properties = find_peaks(filtered_Y, height=10)
    peak_heights = properties['peak_heights']

    # Get top 5 peaks
    if len(peaks) > 5:
        top_5_indices = np.argsort(peak_heights)[-5:][::-1]  # Indices of top 5 peaks
        peaks = peaks[top_5_indices]
        peak_heights = peak_heights[top_5_indices]
    
    # Sort the top 5 peaks in chronological order (by their indices)
    sorted_indices = np.argsort(peaks)
    peaks = peaks[sorted_indices]
    peak_heights = peak_heights[sorted_indices]

    return [filtered_freq[i] for i in peaks], peak_heights

def get_fft_area(freq, filtered_Y):
    alpha_mask = (freq>8) & (freq<13)
    beta_mask = (freq>13) & (freq<30)
    theta_mask = (freq>4) & (freq<8)
    delta_mask = (freq>0.5) & (freq<4)
    gamma_mask = (freq>30) & (freq<50)

    frequency_masks = [alpha_mask, beta_mask, theta_mask, delta_mask, gamma_mask]

    return [trapezoid(filtered_Y[mask], dx=1) for mask in frequency_masks]


def find_max_electrode(data: pd.DataFrame):
    mean_dict = {
        'RAW_AF7': float(data['RAW_AF7'].abs().max() - data['RAW_AF7'].abs().min()), 
        'RAW_AF8': float(data['RAW_AF8'].abs().max() - data['RAW_AF8'].abs().min()), 
        'RAW_TP9': float(data['RAW_TP9'].abs().max() - data['RAW_TP9'].abs().min()), 
        'RAW_TP10': float(data['RAW_TP10'].abs().max() - data['RAW_TP10'].abs().min())
        }
    
    return max(mean_dict, key=mean_dict.get) 

def feature_extraction(all_data):
    feature_data = []

    # for each file 
    for i, data in enumerate(all_data):
        # initialize lists
        peaks_list = []
        amplitude_list = []
        area_list = []

        # extract data from electrodes USING MAX ELECTRODE RAW DATA
        channel = find_max_electrode(data)
        freq, y = get_fft(data, channel)
        peaks, peak_heights = get_peaks(freq, y) # type: ignore
        area = get_fft_area(freq, y)

        peaks_list.extend(peaks)
        amplitude_list.extend(peak_heights)
        area_list.extend(area)

        data_info = peaks_list + amplitude_list + area_list
        feature_data.append(data_info)

    # Convert to DataFrame
    feature_data = pd.DataFrame(feature_data)

    return feature_data

def plot_samples():
    pass

all_features_math = feature_extraction(all_math_data)
all_features_drawing = feature_extraction(all_drawing_data)

# Add labels to the data
all_features_math['label'] = 0
all_features_drawing['label'] = 1

### Envolope the Alpha, Beta, Theta, Gamma Curves

## Exploratory Data Analysis

### Splitting Dateset into the Training and Test Sets

### Feature Scaling

### Principal Component Analysis (PCA)

## Building Machine Learning Models

### Classification: Support Vector Machine (SVM)

In [4]:
# from sklearn.svm import SVC

# classifier = SVC(kernel = 'rbf', random_state = 0) // try differernals and remove the random_state maybe
# classifier.fit(X_train, y_train)

### SVM: Confusion Matrix

In [3]:
# from sklearn.metrics import confusion_matrix, accuracy_score
# cm = confusion_matrix(y_test, y_pred)
# print(cm)
# print("Your Model Accuracy is=", accuracy_score(y_test, y_pred)*100, "%")

### SVM: Prediction a new Result

### Neural Network (ANN) Model